In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from src.model.model import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [2]:
def select_inout(dftest):
    # 결측치 처리
    for strname in list(dftest.columns):
        dftest[strname]=dftest[strname].fillna((dftest[strname].ffill()+\
                                                dftest[strname].bfill())/2)
        
    # 입출력지정하기위해 target col 추가
    time_delta = 1
    dftest["target"]=list(dftest["pm10_value"].values[time_delta:])+[0]*time_delta
    dftest = dftest[:-(time_delta)]

    # 스케일링
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(dftest)
    scaled_df = pd.DataFrame(scaled_data, columns=dftest.columns)

    # 데이터지정
    data = np.array(scaled_df.drop(["pm25_value"],axis=1))

    X = data[:,:-1]
    y = data[:,-1]
    return X,y

In [189]:
# 파일 로드 
df = pd.read_csv("dataset/merged_df.csv") 
df = df[df["date_time"]>"2023-08-25 00:00:00"] # 기상데이터가 있는 기간부터

In [190]:
r2_score_lst = []
for pos_num in range(1,13):
    dftest = df[df["measure_position_id"]==pos_num]
    dftest = dftest.drop(["date_time","measure_position_id","type","name",
                        "latitude","longitude","district_code"],axis=1)
    dftest = dftest[:-2]
    
    X,y = select_inout(dftest)

    # 데이터 분할
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Random Forest 회귀 모델 초기화
    rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)

    # 모델 학습
    rf_regressor.fit(X_train, y_train)

    # 테스트 데이터에 대한 예측
    y_pred = rf_regressor.predict(X_test)

    # 예측 결과 평가 (예: 평균 제곱 오차)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    r2_score_lst.append(r2)
    print(f"R2 Score: {r2}")
    
print(sum(r2_score_lst)/len(r2_score_lst))

R2 Score: 0.7641756018542857
R2 Score: 0.7759790981376173
R2 Score: 0.6352679236449844
R2 Score: 0.5934343620680118
R2 Score: 0.6001121383704623
R2 Score: 0.49178457451995694
R2 Score: 0.739137721796626
R2 Score: 0.5140077631466398
R2 Score: 0.5400031641974268
R2 Score: 0.48511494666903365
R2 Score: 0.766190853724932
R2 Score: 0.5556977624945063
0.621742159218707
